# ISP.MI - Algorithmic Trading Analysis

**Generated:** 2026-02-08 15:20:42

## Overview

This notebook demonstrates a complete algorithmic trading workflow using the **algoshort** library.

### Workflow Steps:
1. **Data Acquisition** - Download historical OHLC data from Yahoo Finance
2. **Relative Price Calculation** - Calculate performance relative to FTSE MIB benchmark
3. **Signal Generation** - Generate trading signals using multiple strategies
4. **Signal Combination** - Combine signals with entry/exit/direction logic
5. **Returns Calculation** - Calculate P&L and equity curves
6. **Stop Loss Calculation** - Implement risk management
7. **Position Sizing** - Determine optimal position sizes

### Key Concepts:
- **Relative Prices**: Stock performance vs. market benchmark (isolates alpha)
- **Regime Detection**: Identify bullish/bearish market conditions
- **Signal Combination**: Use direction + entry + exit logic for robust signals

---

## Configuration

### Ticker Settings

In [1]:
# =============================================================================
# TICKER CONFIGURATION
# =============================================================================
TICKER = "ISP.MI"
BENCHMARK = "FTSEMIB.MI"
START_DATE = "2024-01-01"
INITIAL_CAPITAL = 10000

### Imports and Logging Setup

In [2]:
# =============================================================================
# IMPORTS
# =============================================================================
import logging
import pandas as pd
import numpy as np
from datetime import date
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s [%(levelname)7s] %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

print("Imports completed successfully!")

Imports completed successfully!


---

## 1. Data Acquisition

### YFinanceDataHandler

The `YFinanceDataHandler` class provides:
- **Automatic caching** with parquet files (10-50x faster on repeated access)
- **Bulk downloads** with chunking to avoid rate limits
- **Data quality checks** and cleaning

**Key Methods:**
- `download_data()` - Download data for symbols
- `get_ohlc_data()` - Get formatted OHLC data
- `get_info()` - Get company fundamentals

In [3]:
# =============================================================================
# DATA DOWNLOAD
# =============================================================================
from algoshort.yfinance_handler import YFinanceDataHandler

handler = YFinanceDataHandler(cache_dir="./cache", enable_logging=False, chunk_size=30)

print(f"Downloading data for ISP.MI and {BENCHMARK}...")
print(f"Period: {START_DATE} to {date.today()}")

handler.download_data(
    symbols=[TICKER, BENCHMARK],
    start=START_DATE,
    end=date.today().isoformat(),
    interval='1d',
    use_cache=True
)

print("\nDownload complete!")

Period: 2024-01-01 to 2026-02-08



Download complete!


In [4]:
# =============================================================================
# DATA QUALITY CHECK
# =============================================================================
print("Data Quality Summary:")
print("=" * 60)

summary = handler.list_available_data()
for symbol, info in summary.items():
    print(f"\n{symbol}:")
    print(f"  Rows: {info['rows']}")
    print(f"  Date Range: {info['date_range']}")
    print(f"  Missing Values: {info['missing_values']}")

Data Quality Summary:

ISP.MI:
  Rows: 2566
  Date Range: 2016-01-04 to 2026-02-06
  Missing Values: 0

FTSEMIB.MI:
  Rows: 2564
  Date Range: 2016-01-04 to 2026-02-05
  Missing Values: 0


In [5]:
# =============================================================================
# GET OHLC DATA
# =============================================================================
df = handler.get_ohlc_data(TICKER)
bmk = handler.get_ohlc_data(BENCHMARK)
df['fx'] = 1

print(f"ISP.MI Data Shape: {df.shape}")
print(f"{BENCHMARK} Data Shape: {bmk.shape}")
print(f"\nDate Range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
print(f"\nFirst 5 rows:")
df.head()

ISP.MI Data Shape: (2566, 6)
FTSEMIB.MI Data Shape: (2564, 5)

Date Range: 2016-01-04 to 2026-02-06

First 5 rows:


,date,open,high,low,close,fx
0,2016-01-04 00:00:00+01:00,1.513720,1.517691,1.479972,1.479972,1
1,2016-01-05 00:00:00+01:00,1.490890,1.508757,1.481957,1.508757,1
2,2016-01-06 00:00:00+01:00,1.502802,1.508757,1.468060,1.468060,1
3,2016-01-07 00:00:00+01:00,1.434312,1.457142,1.401556,1.443245,1
4,2016-01-08 00:00:00+01:00,1.446223,1.474016,1.438282,1.438282,1


In [6]:
# =============================================================================
# COMPANY INFORMATION
# =============================================================================
info = handler.get_info(TICKER)

print(f"Company: {info.get('longName', 'N/A')}")
print(f"Sector: {info.get('sector', 'N/A')}")
print(f"Industry: {info.get('industry', 'N/A')}")
print(f"Market Cap: {info.get('marketCap', 0):,.0f}")
print(f"P/E Ratio: {info.get('trailingPE', 'N/A')}")
print(f"Dividend Yield: {info.get('dividendYield', 0)*100:.2f}%" if info.get('dividendYield') else "Dividend Yield: N/A")

Company: Intesa Sanpaolo S.p.A.
Sector: Financial Services
Industry: Banks - Regional
Market Cap: 103,726,440,448
P/E Ratio: 10.466667
Dividend Yield: 637.00%


---

## 2. Relative Price Calculation

### OHLCProcessor

**Why Relative Prices?**

Relative prices show how a stock performs **compared to the market**:
- **Absolute Price**: Stock goes up 5% -> Could be market rally (beta)
- **Relative Price**: Stock goes up 5% vs market 2% -> Stock outperforms (alpha)

**Calculation:**
```
Relative Price = Stock Price / Benchmark Price (rebased to 1.0)
```

In [7]:
# =============================================================================
# CALCULATE RELATIVE PRICES
# =============================================================================
from algoshort.ohlcprocessor import OHLCProcessor

processor = OHLCProcessor()
df = processor.calculate_relative_prices(
    stock_data=df,
    benchmark_data=bmk,
    benchmark_column='close',
    digits=4,
    rebase=True
)

print("Relative OHLC columns created:")
print("  ropen  - Relative Open")
print("  rhigh  - Relative High")
print("  rlow   - Relative Low")
print("  rclose - Relative Close")
print(f"\nDataFrame shape: {df.shape}")

2026-02-08 15:25:13 [WARNING] algoshort.ohlcprocessor.OHLCProcessor: Missing benchmark data for 2 rows (0.1%). Values will be forward-filled. This may indicate misaligned date ranges.


Relative OHLC columns created:
  ropen  - Relative Open
  rhigh  - Relative High
  rlow   - Relative Low
  rclose - Relative Close

DataFrame shape: (2566, 10)


Performance Summary:
  Absolute Return: +303.12%
  Relative Return (vs benchmark): +82.41%


In [8]:
# =============================================================================
# VISUALIZE ABSOLUTE VS RELATIVE PERFORMANCE
# =============================================================================
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

abs_normalized = (df['close'] / df['close'].iloc[0]) * 100
rel_normalized = (df['rclose'] / df['rclose'].iloc[0]) * 100

axes[0].plot(df['date'], abs_normalized, 'b-', linewidth=1, label=f'ISP.MI (Absolute)')
axes[0].axhline(y=100, color='gray', linestyle='--', alpha=0.5)
axes[0].set_ylabel('Normalized Price (Base=100)')
axes[0].set_title(f'ISP.MI - Absolute Performance')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(df['date'], rel_normalized, 'g-', linewidth=1, label=f'ISP.MI vs {BENCHMARK} (Relative)')
axes[1].axhline(y=100, color='gray', linestyle='--', alpha=0.5)
axes[1].set_ylabel('Normalized Price (Base=100)')
axes[1].set_xlabel('Date')
axes[1].set_title(f'ISP.MI - Relative Performance vs {BENCHMARK}')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

abs_return = (df['close'].iloc[-1] / df['close'].iloc[0] - 1) * 100
rel_return = (df['rclose'].iloc[-1] / df['rclose'].iloc[0] - 1) * 100
print(f"\nPerformance Summary:")
print(f"  Absolute Return: {abs_return:+.2f}%")
print(f"  Relative Return (vs benchmark): {rel_return:+.2f}%")

---

## 3. Signal Generation

### Trading Signals Overview

| Method | Class | Description |
|--------|-------|-------------|
| **Breakout** | `RegimeBO` | Price breaks above/below N-day high/low |
| **Turtle Trader** | `RegimeBO` | Dual-window breakout system |
| **MA Crossover** | `TripleMACrossoverRegime` | Triple moving average alignment |
| **Floor/Ceiling** | `RegimeFC` | Swing-based regime detection |

**Signal Values:**
- `1` = Bullish / Long
- `0` = Neutral / Flat
- `-1` = Bearish / Short

In [9]:
# =============================================================================
# BREAKOUT SIGNALS
# =============================================================================
from algoshort.regime_bo import RegimeBO

regime_bo = RegimeBO(ohlc_stock=df)
breakout_windows = [20, 50, 100]

print("Generating Breakout Signals:")
print("=" * 60)

for window in breakout_windows:
    df = regime_bo.compute_regime(regime_type='breakout', window=window, relative=True, inplace=True)
    signal_col = f'rbo_{window}'
    counts = df[signal_col].value_counts().to_dict()
    print(f"\n  rbo_{window} ({window}-day breakout):")
    print(f"    Long (1):  {counts.get(1.0, 0):>5} bars ({counts.get(1.0, 0)/len(df)*100:.1f}%)")
    print(f"    Short (-1): {counts.get(-1.0, 0):>5} bars ({counts.get(-1.0, 0)/len(df)*100:.1f}%)")

Generating Breakout Signals:

  rbo_20 (20-day breakout):
    Long (1):   1436 bars (56.0%)
    Short (-1):  1108 bars (43.2%)

  rbo_50 (50-day breakout):
    Long (1):   1449 bars (56.5%)
    Short (-1):  1052 bars (41.0%)

  rbo_100 (100-day breakout):
    Long (1):   1266 bars (49.3%)
    Short (-1):  1178 bars (45.9%)


In [10]:
# =============================================================================
# TURTLE TRADER SIGNALS
# =============================================================================
print("\nGenerating Turtle Trader Signals:")
print("=" * 60)

df = regime_bo.compute_regime(regime_type='turtle', window=50, fast_window=20, relative=True, inplace=True)

signal_col = 'rtt_5020'
counts = df[signal_col].value_counts().to_dict()
print(f"\n  rtt_5020 (Slow=50, Fast=20):")
print(f"    Long (1):   {counts.get(1, 0):>5} bars ({counts.get(1, 0)/len(df)*100:.1f}%)")
print(f"    Neutral (0): {counts.get(0, 0):>5} bars ({counts.get(0, 0)/len(df)*100:.1f}%)")
print(f"    Short (-1): {counts.get(-1, 0):>5} bars ({counts.get(-1, 0)/len(df)*100:.1f}%)")


Generating Turtle Trader Signals:

  rtt_5020 (Slow=50, Fast=20):
    Long (1):    1050 bars (40.9%)
    Neutral (0):   825 bars (32.2%)
    Short (-1):   691 bars (26.9%)


In [11]:
# =============================================================================
# MOVING AVERAGE CROSSOVER SIGNALS
# =============================================================================
from algoshort.regime_ma import TripleMACrossoverRegime

regime_ma = TripleMACrossoverRegime(ohlc_stock=df)
ma_params = {'short': 50, 'medium': 100, 'long': 150}

print("\nGenerating MA Crossover Signals:")
print("=" * 60)
print(f"Parameters: Short={ma_params['short']}, Medium={ma_params['medium']}, Long={ma_params['long']}")

for ma_type in ['sma', 'ema']:
    df = regime_ma.compute_ma_regime(
        ma_type=ma_type,
        short_window=ma_params['short'],
        medium_window=ma_params['medium'],
        long_window=ma_params['long'],
        relative=True,
        inplace=True
    )
    signal_col = f"r{ma_type}_{ma_params['short']}{ma_params['medium']}{ma_params['long']}"
    counts = df[signal_col].value_counts().to_dict()
    print(f"\n  {signal_col} ({ma_type.upper()}):")
    print(f"    Long (1):  {counts.get(1.0, 0):>5} bars ({counts.get(1.0, 0)/len(df)*100:.1f}%)")
    print(f"    Short (-1): {counts.get(-1.0, 0):>5} bars ({counts.get(-1.0, 0)/len(df)*100:.1f}%)")


Generating MA Crossover Signals:
Parameters: Short=50, Medium=100, Long=150

  rsma_50100150 (SMA):
    Long (1):   1641 bars (64.0%)
    Short (-1):   776 bars (30.2%)

  rema_50100150 (EMA):
    Long (1):   2132 bars (83.1%)
    Short (-1):   285 bars (11.1%)


In [12]:
# =============================================================================
# FLOOR/CEILING REGIME DETECTION
# =============================================================================
from algoshort.regime_fc import RegimeFC

regime_fc = RegimeFC(df=df, log_level=logging.WARNING)

print("\nGenerating Floor/Ceiling Regime:")
print("=" * 60)

df = regime_fc.compute_regime(
    relative=True, lvl=3, vlty_n=63, threshold=0.05,
    dgt=3, d_vol=1, dist_pct=0.05, retrace_pct=0.05, r_vol=1.0
)

if 'rrg' in df.columns:
    counts = df['rrg'].value_counts().to_dict()
    print(f"\n  rrg (Floor/Ceiling Regime):")
    print(f"    Bullish (1):  {counts.get(1.0, 0):>5} bars ({counts.get(1.0, 0)/len(df)*100:.1f}%)")
    print(f"    Bearish (-1): {counts.get(-1.0, 0):>5} bars ({counts.get(-1.0, 0)/len(df)*100:.1f}%)")


Generating Floor/Ceiling Regime:

  rrg (Floor/Ceiling Regime):
    Bullish (1):   1518 bars (59.2%)
    Bearish (-1):   946 bars (36.9%)


In [13]:
# =============================================================================
# SIGNAL SUMMARY
# =============================================================================
signal_columns = [col for col in df.columns
    if any(col.startswith(prefix) for prefix in ['rbo_', 'rtt_', 'rsma_', 'rema_', 'rrg'])
    and not any(kw in col for kw in ['short', 'medium', 'long', '_ch'])]
signal_columns = [x for x in signal_columns if x != "rrg_ch"]

print(f"\nTotal Signals Generated: {len(signal_columns)}")
print("=" * 60)
for sig in signal_columns:
    print(f"  * {sig}")


Total Signals Generated: 11
  * rbo_20
  * rbo_50
  * rbo_100
  * rtt_5020
  * rsma_50100
  * rsma_100150
  * rsma_50100150
  * rema_50100
  * rema_100150
  * rema_50100150
  * rrg


---

## 4. Signal Combination

### Grid Search

**Why Combine Signals?**

Individual signals can be noisy. Combining them provides:
- **Direction Filter**: Only trade in direction of overall trend (floor/ceiling)
- **Entry Timing**: Use momentum signals for entry timing
- **Exit Timing**: Use different signal for exit

In [14]:
# =============================================================================
# GRID SEARCH - TEST ALL COMBINATIONS
# =============================================================================
from algoshort.combiner import SignalGridSearch

direction_col = 'rrg'
entry_exit_signals = [x for x in signal_columns if x != direction_col]

print("Running Grid Search (All Entry/Exit Combinations):")
print("=" * 60)

searcher = SignalGridSearch(
    df=df.copy(),
    available_signals=entry_exit_signals,
    direction_col=direction_col
)

results = searcher.run_grid_search_parallel(
    allow_flips=True,
    require_regime_alignment=True,
    n_jobs=-1,
    backend='multiprocessing'
)

df = searcher.df

Running Grid Search (All Entry/Exit Combinations):
✓ Validation passed: All 10 signals found in dataframe

GRID GENERATION
Direction column: rrg
Available signals for entry/exit: 10
Signals: ['rbo_20', 'rbo_50', 'rbo_100', 'rtt_5020', 'rsma_50100', 'rsma_100150', 'rsma_50100150', 'rema_50100', 'rema_100150', 'rema_50100150']

Total combinations to test: 100
  = 10 entry signals × 10 exit signals

RUNNING PARALLEL GRID SEARCH
Direction column: rrg
Allow flips: True
Require regime alignment: True
Backend: multiprocessing
Parallel jobs: 8 cores

Processing 100 combinations in parallel...


Testing combinations:   0%|          | 0/100 [00:00<?, ?it/s]

Testing combinations:   1%|          | 1/100 [00:03<06:07,  3.71s/it]

Testing combinations:  25%|██▌       | 25/100 [00:03<00:08,  9.08it/s]

Testing combinations:  97%|█████████▋| 97/100 [00:04<00:00, 38.30it/s]

Testing combinations: 100%|██████████| 100/100 [00:04<00:00, 23.47it/s]


Adding combined signal columns to main dataframe...

GRID SEARCH COMPLETE
Successful combinations: 100
Failed combinations: 0


In [15]:
# =============================================================================
# GRID SEARCH RESULTS
# =============================================================================
print("\nGrid Search Results:")
print("=" * 60)
print(f"Total Combinations Tested: {len(results)}")
print(f"Successful: {results['success'].sum()}")
print(f"Failed: {(~results['success']).sum()}")

print("\nTop 10 Most Active Combinations:")
top_results = results.nlargest(10, 'total_trades')[[
    'combination_name', 'total_trades', 'long_trades', 'short_trades', 'long_pct', 'short_pct'
]]
print(top_results.to_string(index=False))


Grid Search Results:
Total Combinations Tested: 100
Successful: 100
Failed: 0

Top 10 Most Active Combinations:
          combination_name  total_trades  long_trades  short_trades  long_pct  short_pct
       rema_100150__rbo_20           426          249           177 41.582229  18.784100
        rsma_50100__rbo_20           411          247           164 38.815277  18.978956
       rsma_100150__rbo_20           409          222           187 40.997662  18.199532
        rema_50100__rbo_20           403          237           166 42.556508  19.875292
 rema_50100__rsma_50100150           367          150           214 46.531567  20.810600
rema_100150__rsma_50100150           364          147           214 46.141855  17.770850
           rbo_100__rbo_20           343          163           180 34.450507  20.537802
 rema_50100__rema_50100150           336           47           289 49.805144  15.393609
rsma_100150__rsma_50100150           332          115           217 44.544037  15.8612

---

## 5. Returns Calculation

### ReturnsCalculator

**Calculates for each signal:**
- `{signal}_chg1D`: Daily price change x position
- `{signal}_PL_cum`: Cumulative P&L
- `{signal}_returns`: Percentage returns
- `{signal}_cumul`: Cumulative returns

In [16]:
# =============================================================================
# CALCULATE RETURNS
# =============================================================================
from algoshort.returns import ReturnsCalculator

returns_calc = ReturnsCalculator(
    ohlc_stock=df,
    open_col="open", high_col="high", low_col="low", close_col="close",
    relative_prefix="r"
)

print("Calculating Returns for All Signals:")
print("=" * 60)

df = returns_calc.get_returns_multiple(df=df, signals=signal_columns, relative=True, n_jobs=-1, verbose=False)

print(f"\nReturn columns created for {len(signal_columns)} signals")

Calculating Returns for All Signals:



Return columns created for 11 signals


In [17]:
# =============================================================================
# RETURNS SUMMARY
# =============================================================================
print("\nCumulative Returns by Signal:")
print("=" * 60)

cumul_returns = {}
for sig in signal_columns:
    cumul_col = f'{sig}_cumul'
    if cumul_col in df.columns:
        cumul_returns[sig] = df[cumul_col].iloc[-1] * 100

sorted_returns = sorted(cumul_returns.items(), key=lambda x: x[1], reverse=True)

for sig, ret in sorted_returns:
    print(f"  {sig}: {ret:+.2f}%")


Cumulative Returns by Signal:
  rema_50100150: +97.92%
  rrg: +96.49%
  rsma_100150: +45.46%
  rsma_50100150: +44.09%
  rbo_20: +21.67%
  rema_50100: +13.17%
  rtt_5020: -9.36%
  rema_100150: -13.84%
  rsma_50100: -26.13%
  rbo_100: -42.94%
  rbo_50: -46.19%


---

## 6. Stop Loss Calculation

### StopLossCalculator

**Available Methods:**
| Method | Description |
|--------|-------------|
| `atr_stop_loss` | ATR-based (volatility-adjusted) |
| `fixed_percentage_stop_loss` | Fixed % below/above entry |
| `breakout_channel_stop_loss` | N-day high/low |
| `moving_average_stop_loss` | MA-based stops |

In [18]:
# =============================================================================
# CALCULATE STOP LOSSES
# =============================================================================
from algoshort.stop_loss import StopLossCalculator

sl_calc = StopLossCalculator(df)

print("Calculating ATR-Based Stop Losses:")
print("=" * 60)

for signal in signal_columns:
    df = sl_calc.atr_stop_loss(signal=signal, window=14, multiplier=2.0)
    sl_calc.data = df

sl_cols = [col for col in df.columns if col.endswith('_stop_loss')]
print(f"\nStop loss columns created: {len(sl_cols)}")

Calculating ATR-Based Stop Losses:

Stop loss columns created: 11


---

## 7. Position Sizing

### PositionSizing

**Position Sizing Strategies:**

| Strategy | Description | Risk Profile |
|----------|-------------|-------------|
| `equal_weight` | Fixed % of capital | Moderate |
| `constant` | Fixed risk per trade | Moderate |
| `concave` | Reduce risk as drawdown increases | Conservative |
| `convex` | Increase risk when winning | Aggressive |

In [19]:
# =============================================================================
# POSITION SIZING
# =============================================================================
from algoshort.position_sizing import PositionSizing, run_position_sizing_parallel

sizer = PositionSizing(
    tolerance=-0.10, mn=-0.0025, mx=-0.05,
    equal_weight=0.05, avg=0.03, lot=1,
    initial_capital=INITIAL_CAPITAL
)

print(f"Position Sizing Configuration:")
print("=" * 60)
print(f"  Initial Capital: {INITIAL_CAPITAL:,}")
print(f"  Equal Weight: 5% per position")
print(f"  Risk Range: 0.25% - 5% per trade")
print(f"  Max Drawdown Tolerance: 10%")

print("\nCalculating Position Sizes...")

df = run_position_sizing_parallel(
    sizer=sizer, df=df, signals=signal_columns,
    chg_suffix="_chg1D_fx", sl_suffix="_stop_loss",
    close_col='close', n_jobs=-1, verbose=5
)

Position Sizing Configuration:
  Initial Capital: 10,000
  Equal Weight: 5% per position
  Risk Range: 0.25% - 5% per trade
  Max Drawdown Tolerance: 10%

Calculating Position Sizes...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


[Parallel(n_jobs=-1)]: Done   2 out of  11 | elapsed:    2.2s remaining:    9.8s
[Parallel(n_jobs=-1)]: Done   5 out of  11 | elapsed:    2.3s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done   8 out of  11 | elapsed:    2.4s remaining:    0.9s


[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:    3.7s finished


In [20]:
# =============================================================================
# EQUITY CURVE ANALYSIS
# =============================================================================
print("\nEquity Curve Summary:")
print("=" * 60)

equity_cols = [col for col in df.columns if '_equity_equal' in col]

equity_results = []
for col in equity_cols:
    signal_name = col.replace('_equity_equal', '')
    final_equity = df[col].iloc[-1]
    total_return = (final_equity / INITIAL_CAPITAL - 1) * 100
    max_equity = df[col].max()
    max_drawdown = (df[col].min() - max_equity) / max_equity * 100
    equity_results.append({
        'Signal': signal_name,
        'Final Equity': final_equity,
        'Total Return': total_return,
        'Max Drawdown': max_drawdown
    })

equity_df = pd.DataFrame(equity_results).sort_values('Total Return', ascending=False)
print(equity_df.to_string(index=False))


Equity Curve Summary:
       Signal  Final Equity  Total Return  Max Drawdown
rema_50100150    10513.7409      5.137409     -5.942239
          rrg    10387.9070      3.879070     -5.393669
   rema_50100    10212.7237      2.127237     -5.023305
rsma_50100150    10205.3724      2.053724     -3.837422
  rsma_100150    10184.9683      1.849683     -4.498902
       rbo_20    10048.3578      0.483578     -3.516486
  rema_100150    10010.6291      0.106291     -4.638583
     rtt_5020     9892.0830     -1.079170     -2.703951
   rsma_50100     9876.2693     -1.237307     -5.622650
      rbo_100     9698.7805     -3.012195     -6.269806
       rbo_50     9654.2881     -3.457119     -6.032010


In [21]:
# =============================================================================
# VISUALIZE EQUITY CURVES
# =============================================================================
top_signals = equity_df.head(3)['Signal'].tolist()

fig, ax = plt.subplots(figsize=(14, 6))

for signal in top_signals:
    equity_col = f'{signal}_equity_equal'
    ax.plot(df['date'], df[equity_col], label=signal, linewidth=1)

ax.axhline(y=INITIAL_CAPITAL, color='gray', linestyle='--', alpha=0.5, label='Initial Capital')

ax.set_xlabel('Date')
ax.set_ylabel('Equity')
ax.set_title(f'ISP.MI - Top 3 Strategy Equity Curves (Equal Weight)')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))

plt.tight_layout()
plt.show()

---

## 8. Final Summary

### Complete Analysis Results

In [22]:
# =============================================================================
# FINAL SUMMARY
# =============================================================================
print("=" * 70)
print(f"FINAL ANALYSIS SUMMARY - ISP.MI")
print("=" * 70)

print(f"\n1. DATA:")
print(f"   Ticker: ISP.MI")
print(f"   Benchmark: {BENCHMARK}")
print(f"   Period: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
print(f"   Trading Days: {len(df):,}")

abs_return = (df['close'].iloc[-1] / df['close'].iloc[0] - 1) * 100
rel_return = (df['rclose'].iloc[-1] / df['rclose'].iloc[0] - 1) * 100
print(f"\n2. BUY & HOLD PERFORMANCE:")
print(f"   Absolute Return: {abs_return:+.2f}%")
print(f"   Relative Return (vs {BENCHMARK}): {rel_return:+.2f}%")

best_strategy = equity_df.iloc[0]
print(f"\n3. BEST STRATEGY:")
print(f"   Signal: {best_strategy['Signal']}")
print(f"   Total Return: {best_strategy['Total Return']:+.2f}%")
print(f"   Final Equity: {best_strategy['Final Equity']:,.0f}")

print(f"\n4. DATAFRAME:")
print(f"   Shape: {df.shape}")
print(f"   Memory Usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

ohlc_cols = len([c for c in df.columns if c in ['date', 'open', 'high', 'low', 'close', 'fx']])
rel_cols = len([c for c in df.columns if c.startswith('r') and c[1:] in ['open', 'high', 'low', 'close']])
signal_cols_count = len(signal_columns)
return_cols = len([c for c in df.columns if any(k in c for k in ['_chg1D', '_PL_cum', '_returns', '_cumul'])])
sl_cols_count = len([c for c in df.columns if '_stop_loss' in c])
equity_cols_count = len([c for c in df.columns if '_equity_' in c])

print(f"\n5. COLUMNS BREAKDOWN:")
print(f"   OHLC: {ohlc_cols}")
print(f"   Relative OHLC: {rel_cols}")
print(f"   Signals: {signal_cols_count}")
print(f"   Returns: {return_cols}")
print(f"   Stop Losses: {sl_cols_count}")
print(f"   Equity Curves: {equity_cols_count}")

print("\n" + "=" * 70)
print("ANALYSIS COMPLETE!")
print("=" * 70)

FINAL ANALYSIS SUMMARY - ISP.MI

1. DATA:
   Ticker: ISP.MI
   Benchmark: FTSEMIB.MI
   Period: 2016-01-04 to 2026-02-06
   Trading Days: 2,566

2. BUY & HOLD PERFORMANCE:
   Absolute Return: +303.12%
   Relative Return (vs FTSEMIB.MI): +82.41%

3. BEST STRATEGY:
   Signal: rema_50100150
   Total Return: +5.14%
   Final Equity: 10,514

4. DATAFRAME:
   Shape: (2566, 342)
   Memory Usage: 6.70 MB

5. COLUMNS BREAKDOWN:
   OHLC: 6
   Relative OHLC: 4
   Signals: 11
   Returns: 77
   Stop Losses: 11
   Equity Curves: 44

ANALYSIS COMPLETE!


In [23]:
# =============================================================================
# EXPORT RESULTS
# =============================================================================
import os

output_dir = "./results"
os.makedirs(output_dir, exist_ok=True)

# Export equity summary
equity_df.to_csv(f"{output_dir}/{TICKER.replace('.', '_')}_equity.csv", index=False)

# Export latest signals
latest = df.tail(1)[['date'] + signal_columns].copy()
latest['ticker'] = TICKER
latest.to_csv(f"{output_dir}/{TICKER.replace('.', '_')}_signals.csv", index=False)

# Export grid search results
results.to_csv(f"{output_dir}/{TICKER.replace('.', '_')}_grid_search.csv", index=False)

print(f"Results exported to {output_dir}/")
print(f"  - {TICKER.replace('.', '_')}_equity.csv")
print(f"  - {TICKER.replace('.', '_')}_signals.csv")
print(f"  - {TICKER.replace('.', '_')}_grid_search.csv")

Results exported to ./results/
  - ISP_MI_equity.csv
  - ISP_MI_signals.csv
  - ISP_MI_grid_search.csv
